In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import pickle
import numpy as np
from sklearn import preprocessing
from tqdm import tqdm_notebook
from scipy.stats import skew
from sklearn.metrics import f1_score
plt.style.use(["dark_background"])
%matplotlib inline


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.chdir("../")
os.getcwd()

'/project/at082-group17'

### Loading dataset

In [3]:
## load 'data'
dataNameList = ['201801', '201802', '201803' ,'201804']
dataCount = []
df_all = pd.DataFrame()
for name in dataNameList:
    with open('data/pickle/' + name + '.p', "rb") as file:
        loadData = pickle.load(file)
        dataCount.append(len(loadData.index))
        df_all = pd.concat((df_all, loadData), axis = 0, ignore_index = False)
print('data count ' + str(len(df_all.index)))

data count 33897460


## 取得收信量

In [4]:
df_temp = df_all[["recipientSn", "mailAddress"]].groupby("mailAddress").count().reset_index()

df_temp.columns = ["mailAddress", "mailAddress_count"]
df_temp = df_temp.sort_values(by=['mailAddress_count'], ascending = False).reset_index()

#df_all = df_all.merge(df_temp, how="left", on="mailAddress")

In [5]:
traget = []
for i in range(10):
    traget.append(df_temp['mailAddress'][i])
traget

['70a0b4aface445d4c01fcda547e15043d5b0a97f74e20117aab3168a7aa76b7e146f15d46b48a3b4a7dd60fcd154c7fb42b61a84ac1136a816800e3afba9c481',
 '661c3a87c3182435a34df88a525014528ad7808ac2487b4af0a060f57c86323b126e8459b1fda29e5a94d3ea663cbb1b53411068ac2ffe25daff061b90e9a3eb',
 'f19e420d5f2dd884d3242d4ff352d598057bee7878e7f09ee72e0ff35952de2d64d40a7684b67eb8936bcac4e69b5b41efba75a2ebd9eeef76525c31cfb50abe',
 '63b80e02df1855786309304e8bf243e25080fc03e4cf6384f37eb624bce563a8785a83d2359c92fc3181ad38f8bb818a013e1eae219d8ef43256dbeb5656ae42',
 'bda88f56fd104ee2d99b73bbab4e4a007801d23ccfd8b1e166f3d314731044d533712f3e23764f163ad3928e61a4f137570b89126920b7f88978a97d7281d9d7',
 'f8b226279193bdfea949b4e6d603a2d4f6e9dcf1d6a734bf5e329fc0ce7ceebdb6348a95b08bedc4b40dc6aec1d81f6ff00902bb5d9f6ed07e0b258d436c46fb',
 '305d859a70c6238e891921db50b97cc5acd04879e4c0e89621db190cce312abdaccbcd0b12add7abc57a11695b9b6edad3a59f2696b67b528a1e058ebe8e3ddb',
 'd22883701224a5debd82d5274c37976618058a1b159088bf0d7649b9bcf32261815

In [40]:
df_t1 = df_all[df_all['mailAddress'] == traget[1]]
df_t1.head(5)

,recipientSn,campaignSn,mailAddress,domain,openedFlag,fromMailAddress,scheduledDate,domain_count,dayofweek,time
73366,00000000606f8cfb0160b3ae47bc51fd,00000000609630da0160a18969936daf,661c3a87c3182435a34df88a525014528ad7808ac2487b...,gmail.com,False,event@brain.com.tw,2018-01-02 05:43:00,2943558,1,0.074108
134955,00000000606f8cfb0160b4612c5a603d,00000000609630da0160a1cb646933d9,661c3a87c3182435a34df88a525014528ad7808ac2487b...,gmail.com,False,askme@brain.com.tw,2018-01-02 08:58:23,2943558,1,-0.702101
149753,00000000606f8cfb0160b462658f367a,00000000609630da0160a19457007463,661c3a87c3182435a34df88a525014528ad7808ac2487b...,gmail.com,False,lecture@brain.com.tw,2018-01-02 09:00:00,2943558,1,-0.707107
209730,00000000606f8cfb0160b4d08bd12fb9,00000000609630da0160a11e2e416bd2,661c3a87c3182435a34df88a525014528ad7808ac2487b...,gmail.com,True,ivy@wemoscooter.com,2018-01-02 11:00:00,2943558,1,-0.965926
419326,00000000606f8cfb0160b8ad59e27d92,00000000609630da0160b5f95ade25e2,661c3a87c3182435a34df88a525014528ad7808ac2487b...,gmail.com,False,askme@brain.com.tw,2018-01-03 05:00:00,2943558,2,0.258819


In [41]:
## save data as pickle format
with open('data/pickle/' + 'df_t1' + '.p', "wb") as file:
    pickle.dump(df_t1, file)

## Standardization
正規化 將數值轉為[0,1] or [-1,1] <br>
只對數值類型的做

In [26]:
trainCount = df_t1[df_t1['scheduledDate'].apply(lambda x :x.month) <= 3].count()
trainCount = trainCount[0]
df_t1[ 'time' ] = preprocessing.scale( df_t1[ 'time'] ,with_mean=True,with_std=True)
df_t1['dayofweek'] = df_t1['dayofweek'].astype( str )
df_t1 = df_t1.drop(['domain','recipientSn', 'campaignSn', 'mailAddress', 'fromMailAddress', 'scheduledDate', 'domain_count'], axis = 1)
X = df_t1.drop(['openedFlag'], axis = 1)
y = df_t1[ 'openedFlag' ]

x_train = X[:trainCount]
y_train = y[:trainCount]
x_test = X[trainCount:]
y_test = y[trainCount:]
df_t1.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,openedFlag,dayofweek,time
73366,False,1,0.644622
134955,False,1,-0.744149
149753,False,1,-0.753105
209730,True,1,-1.216176
419326,False,2,0.975101


### Build model

In [27]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [28]:
model = Sequential()
model.add(Embedding(10000, 128))
model.add(LSTM(150))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               167400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151       
Total params: 1,447,551
Trainable params: 1,447,551
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [30]:
model.fit(x_train, y_train, batch_size = 32, epochs = 10)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
214/214 [==============================] - 1s 4ms/step - loss: 0.6543 - acc: 0.6215
Epoch 2/10
214/214 [==============================] - 0s 206us/step - loss: 0.6571 - acc: 0.6449
Epoch 3/10
214/214 [==============================] - 0s 199us/step - loss: 0.6462 - acc: 0.6449
Epoch 4/10
214/214 [==============================] - 0s 188us/step - loss: 0.6391 - acc: 0.6449
Epoch 5/10
214/214 [==============================] - 0s 200us/step - loss: 0.6354 - acc: 0.6449
Epoch 6/10
214/214 [==============================] - 0s 219us/step - loss: 0.6364 - acc: 0.6589
Epoch 7/10
214/214 [==============================] - 0s 226us/step - loss: 0.6316 - acc: 0.6869
Epoch 8/10
214/214 [==============================] - 0s 192us/step - loss: 0.6290 - acc: 0.6869
Epoch 9/10
214/214 [==============================] - 0s 192us/step - loss: 0.6276 - acc: 0.6869
Epoch 10/10
214/214 [==============================] - 0s 201us/step - loss: 0.6292 - acc: 0.6869


In [31]:
score = model.evaluate(x_test, y_test)

77/77 [==============================] - 0s 2ms/step


In [32]:
score

[0.6213557263473412, 0.7012987051691327]

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [38]:
threshold_LSTM = 0.1

y_pred_proba_LSTM = model.predict_proba(x_test)
y_pred_LSTM = (y_pred_proba_LSTM >= threshold_LSTM).astype('int')
print("acc = " + str(accuracy_score(y_test, y_pred_LSTM)))
print("recall = " + str(recall_score(y_test, y_pred_LSTM)))
print("f1 = " + str(f1_score(y_test, y_pred_LSTM)))

acc = 0.36363636363636365
recall = 1.0
f1 = 0.5333333333333333


In [30]:
print('測試資料的 loss:', score[0])
print('測試資料的正確率:', score[1])

測試資料的 loss: 0.37745190639885107
測試資料的正確率: 0.8673469387755102
